In [1]:
import itertools
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import numpy_groupies as npg
import pandas as pd
import scipy.stats as s
import scipy.special as ss
import sortedcontainers as sc
import os
import tqdm
import matplotlib

In [2]:
os.environ["CAPY_REF_FA"] = "/home/opriebe/data/ref/hg19/Homo_sapiens_assembly19.fasta"
import hapaseg.coverage_MCMC as mcmc_cov
import hapaseg.NB_coverage_MCMC as nb_cov
from capy import mut, seq

Cannot find path to gnomAD index; gnomAD functionality disabled.


In [3]:
import hapaseg.coverage_DP as dp_cov

Load MCMC trace over SNP DP cluster assignments. Still looking at only the high purity exome

In [4]:
colors = mpl.cm.get_cmap("tab10").colors

### Load coverage

In [5]:
Cov = pd.read_csv("exome/6_C1D1_META.cov", sep = "\t", names = ["chr", "start", "end", "covcorr", "covraw"], low_memory=False)
Cov["chr"] = mut.convert_chr(Cov["chr"])
Cov = Cov.loc[Cov["chr"] != 0]
Cov["start_g"] = seq.chrpos2gpos(Cov["chr"], Cov["start"])
Cov["end_g"] = seq.chrpos2gpos(Cov["chr"], Cov["end"])

### Load covariates

In [6]:
Cov["C_len"] = Cov["end"] - Cov["start"] + 1

# load repl timing
F = pd.read_pickle("covars/GSE137764_H1.hg19_liftover.pickle")

# map targets to RT intervals
tidx = mut.map_mutations_to_targets(Cov.rename(columns = { "start" : "pos" }), F, inplace = False)
Cov.loc[tidx.index, "C_RT"] = F.iloc[tidx, 3:].mean(1).values

# z-transform
Cov["C_RT_z"] = (lambda x : (x - np.nanmean(x))/np.nanstd(x))(np.log(Cov["C_RT"] + 1e-20))

#load GC content
B = pd.read_pickle("covars/GC.pickle")
Cov = Cov.merge(B.rename(columns = { "gc" : "C_GC" }), left_on = ["chr", "start", "end"], right_on = ["chr", "start", "end"], how = "left")
Cov["C_GC_z"] = (lambda x : (x - np.nanmean(x))/np.nanstd(x))(np.log(Cov["C_GC"] + 1e-20))

In [7]:
clust = np.load("exome/6_C1D1_META.DP_clusts.auto_ref_correct.overdispersion92.no_phase_correct.npz")

#### Load chromosome boundary coordinates

In [8]:
allelic_segs = pd.read_pickle("exome/6_C1D1_META.allelic_segs.auto_ref_correct.overdispersion92.no_phase_correct.pickle")
chrbdy = allelic_segs.dropna().loc[:, ["start", "end"]]
chr_ends = chrbdy.loc[chrbdy["start"] != 0, "end"].cumsum()

In [9]:
clust["snps_to_clusters"].shape

(500, 11768)

In [10]:
SNPs = pd.read_pickle("exome/6_C1D1_META.SNPs.pickle")
SNPs["chr"], SNPs["pos"] = seq.gpos2chrpos(SNPs["gpos"])

SNPs["tidx"] = mut.map_mutations_to_targets(SNPs, Cov, inplace = False)

#generate unique clust assignments
clust_u, clust_uj = np.unique(clust["snps_to_clusters"], return_inverse = True)
clust_uj = clust_uj.reshape(clust["snps_to_clusters"].shape)

### load SNPs from single DP iteration

In [11]:
iter_ind = 499
SNPs = pd.read_pickle("exome/6_C1D1_META.SNPs.pickle")
SNPs["chr"], SNPs["pos"] = seq.gpos2chrpos(SNPs["gpos"])

SNPs["tidx"] = mut.map_mutations_to_targets(SNPs, Cov, inplace = False)

#generate unique clust assignments
clust_u, clust_uj = np.unique(clust["snps_to_clusters"], return_inverse = True)
clust_uj = clust_uj.reshape(clust["snps_to_clusters"].shape)

#assign coverage intervals to clusters
Cov_clust_probs = np.zeros([len(Cov), clust_u.max()])

for targ, snp_idx in SNPs.groupby("tidx").indices.items():
    targ_clust_hist = np.bincount(clust_uj[499, snp_idx].ravel(), minlength = clust_u.max())

    Cov_clust_probs[int(targ), :] = targ_clust_hist/targ_clust_hist.sum()

#subset intervals containing SNPs
overlap_idx = Cov_clust_probs.sum(1) > 0
Cov_clust_probs_overlap = Cov_clust_probs[overlap_idx, :]

#prune improbable assignments
Cov_clust_probs_overlap[Cov_clust_probs_overlap < 0.05] = 0
Cov_clust_probs_overlap /= Cov_clust_probs_overlap.sum(1)[:, None]
prune_idx = Cov_clust_probs_overlap.sum(0) > 0
Cov_clust_probs_overlap = Cov_clust_probs_overlap[:, prune_idx]

In [12]:
# import scipy.stats
# assgn_entropy = scipy.stats.entropy(Cov_clust_probs_overlap, axis=1)
# entropy_idx = (assgn_entropy == 0)

#subsetting to only targets that overlap SNPs
Cov_overlap = Cov.loc[overlap_idx, :]

# probablistically assign each ambiguous coverage bin to a cluster
amb_mask= np.max(Cov_clust_probs_overlap, 1) != 1
amb_assgn_probs = Cov_clust_probs_overlap[amb_mask, :]
new_assgn = np.array([np.random.choice(np.r_[:19], p = amb_assgn_probs[i]) for i in range(len(amb_assgn_probs))])
new_onehot = np.zeros((new_assgn.size, new_assgn.max()+1))
new_onehot[np.arange(new_assgn.size), new_assgn] = 1

#subsetting to only targets with hard assignments
#Cov_overlap = Cov_overlap.loc[entropy_idx,:]

r = np.c_[Cov_overlap["covcorr"]]

#making covariate matrix
C = np.c_[np.log(Cov_overlap["C_len"]), Cov_overlap["C_RT_z"], Cov_overlap["C_GC_z"]]

# Pi = Cov_clust_probs_overlap[entropy_idx,:].copy()
Pi = Cov_clust_probs_overlap.copy()
Pi[amb_mask, :] = new_onehot

#dropping Nans
naidx = np.isnan(C[:, 1])
r = r[~naidx]
C = C[~naidx]
Pi = Pi[~naidx]

Cov_overlap = Cov_overlap.iloc[~naidx]

mu = np.log(r.mean()*np.ones([Pi.shape[1], 1])) / 2
beta = np.ones([C.shape[1], 1])
mu_i = np.zeros(C.shape[0])
epsi = np.ones((mu.shape)) *2

Cov_overlap['cluster_assgn'] = np.argmax(Pi, axis=1)

almost all bins are unanimously assigned

In [81]:
nb_mcmc = nb_cov.NB_MCMC(r, C, Pi)

run using stats models for split but not for join

In [82]:
nb_mcmc.run(500)

  0%|          | 0/500 [00:00<?, ?it/s]

starting MCMC coverage segmentation...


 23%|██▎       | 113/500 [04:33<15:36,  2.42s/it]


KeyboardInterrupt: 

In [16]:
len(nb_mcmc.ll_iter)

430

In [34]:
overlap_tst = Cov_overlap.copy()

In [35]:
overlap_tst['segment_ID'] = 0

In [37]:
seg_id_col = overlap_tst.columns.get_loc('segment_ID')

In [39]:
#convert Cov_MCMC object into segment objects to be used by cov_DP

global_seg_counter = 0
for i, clust in enumerate(nb_mcmc.clusters):
    pi_argmax = Pi.argmax(1)
    og_positions = np.where(pi_argmax == i)[0]
    seg_intervals = np.array(clust.F).reshape(-1,2)
    for st, en in seg_intervals:
        seg_ind = og_positions[st:en]
        overlap_tst.iloc[seg_ind, seg_id_col] = global_seg_counter
        global_seg_counter += 1

In [46]:
class Segment:
    def __init__(self, r, C):
        self.r = r
        self.C = C

In [67]:
segment_list = [None] * (overlap_tst.segment_ID.max() + 1)

In [69]:
for ID, seg_df in overlap_tst.groupby('segment_ID'):
    segment_list[ID] = Segment(seg_df['covcorr'], np.c_[np.log(seg_df["C_len"]), seg_df["C_RT_z"], seg_df["C_GC_z"]])

In [57]:
#can be used instead of Pi
overlap_tst.cluster_assgn.values

array([3, 0, 3, ..., 0, 0, 0], dtype=int64)

In [13]:
#loading cov_seg_df
cov_seg_df = pd.read_pickle('./cov_MCMC_df')

In [15]:
beta = np.load('./beta_save.npy')

In [17]:
cov_dp = dp_cov.Cov_DP(cov_seg_df, beta)

AttributeError: module 'sortedcontainers' has no attribute 'Sorted_list'